In [35]:
import pandas as pd
import os
import numpy as np
import openpyxl
import psycopg2
import glob

In [36]:
df = pd.read_excel("customer_payments.xlsx")
df.head()

,payment_id,payment_number,invoice_number,date,payment_mode,amount,bcy_amount,unused_amount,bcy_unused_amount,bcy_bank_charge,...,last_four_digits,type,bcy_refunded_amount,has_attachment,documents,tax_account_id,tax_account_name,tax_amount_withheld,currency_code,branch_name
0,2620362000003131790,1053,INV-JSA000430,2022-11-15,Bank Transfer,0,11519.77,0,0.0,0.0,...,NaN,renewal,0,False,NaN,NaN,NaN,0.0,USD,Jetstream Africa
1,2620362000003131908,1056,INV-JSA000875,2022-11-15,Bank Transfer,0,8903.74,0,0.0,0.0,...,NaN,renewal,0,False,NaN,NaN,NaN,0.0,USD,Jetstream Africa
2,2620362000003146013,1060,INV-JSA000778,2022-11-15,Bank Transfer,0,10000.00,0,0.0,0.0,...,NaN,renewal,0,False,NaN,NaN,NaN,0.0,GHS,Jetstream Africa
3,2620362000003131832,1054,INV-JSA000875,2022-11-14,Bank Transfer,0,13998.01,0,0.0,0.0,...,NaN,renewal,0,False,NaN,NaN,NaN,0.0,USD,Jetstream Africa
4,2620362000003131870,1055,INV-JSA000875,2022-11-14,MoMo Transfer,0,1500.03,0,0.0,0.0,...,NaN,renewal,0,False,NaN,NaN,NaN,0.0,USD,Jetstream Africa


In [37]:
df.columns = [x.lower().replace(" ","_").replace("%","")for x in df.columns]
df.columns

Index(['payment_id', 'payment_number', 'invoice_number', 'date',
       'payment_mode', 'amount', 'bcy_amount', 'unused_amount',
       'bcy_unused_amount', 'bcy_bank_charge', 'account_id', 'account_name',
       'description', 'reference_number', 'customer_id', 'customer_name',
       'last_four_digits', 'type', 'bcy_refunded_amount', 'has_attachment',
       'documents', 'tax_account_id', 'tax_account_name',
       'tax_amount_withheld', 'currency_code', 'branch_name'],
      dtype='object')

In [38]:
df.dtypes

payment_id               int64
payment_number           int64
invoice_number          object
date                    object
payment_mode            object
amount                   int64
bcy_amount             float64
unused_amount            int64
bcy_unused_amount      float64
bcy_bank_charge        float64
account_id               int64
account_name            object
description             object
reference_number        object
customer_id              int64
customer_name           object
last_four_digits       float64
type                    object
bcy_refunded_amount      int64
has_attachment            bool
documents              float64
tax_account_id         float64
tax_account_name        object
tax_amount_withheld    float64
currency_code           object
branch_name             object
dtype: object

In [39]:
replacements = {
    'object' : 'varchar',
    'float64' : 'float',
    'int64' : 'varchar',
    'datetime64[ns]': 'varchar',
    'datetime64': 'timestamp',
    'bool' : 'varchar'
}
replacements

{'object': 'varchar',
 'float64': 'float',
 'int64': 'varchar',
 'datetime64[ns]': 'varchar',
 'datetime64': 'timestamp',
 'bool': 'varchar'}

In [40]:
col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'payment_id varchar, payment_number varchar, invoice_number varchar, date varchar, payment_mode varchar, amount varchar, bcy_amount float, unused_amount varchar, bcy_unused_amount float, bcy_bank_charge float, account_id varchar, account_name varchar, description varchar, reference_number varchar, customer_id varchar, customer_name varchar, last_four_digits float, type varchar, bcy_refunded_amount varchar, has_attachment varchar, documents float, tax_account_id float, tax_account_name varchar, tax_amount_withheld float, currency_code varchar, branch_name varchar'

In [41]:
conn_string = "host= db.app.production.jetstreamafrica.com \
                dbname='jetstream-finance' \
                    user='ujetstreamfinance' password='Smehr5A7FGw4jEX@' "

conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

print('opened database successfully')

opened database successfully


In [42]:
cursor.execute("drop table if exists customer_payments;")

In [43]:
cursor.execute("create table customer_payments \
    (payment_id varchar, payment_number varchar, invoice_number varchar, date varchar, payment_mode varchar, amount varchar, bcy_amount float, unused_amount varchar, bcy_unused_amount float, bcy_bank_charge float, account_id varchar, account_name varchar, description varchar, reference_number varchar, customer_id varchar, customer_name varchar, last_four_digits float, type varchar, bcy_refunded_amount varchar, has_attachment varchar, documents float, tax_account_id varchar, tax_account_name varchar, tax_amount_withheld float, currency_code varchar, branch_name varchar)")

In [44]:
#insert values to tables
#save df to csv
df.to_csv('customer_payments', header=df.columns, index=False, encoding='utf-8')
#open the csv file, save it as an object and upload to db
my_file = open("customer_payments")
print('file opened in memory')

file opened in memory


In [45]:
SQL_STATEMENT = """
COPY customer_payments FROM STDIN WITH
   CSV
   HEADER
   DELIMITER AS ','
"""
cursor.copy_expert(sql = SQL_STATEMENT, file=my_file)
print('file copied to db')

file copied to db


In [46]:
cursor.execute("grant select on table customer_payments to public")
conn.commit()

cursor.close()
print('table customer_payments imported successfully')

table customer_payments imported successfully
